In [ ]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Charger le modèle et le tokenizer
tokenizer = AutoTokenizer.from_pretrained("bakrianoo/t5-arabic-small")
model = AutoModelForSeq2SeqLM.from_pretrained("bakrianoo/t5-arabic-small")

# Charger votre fichier CSV avec questions et réponses
df = pd.read_csv('questions_translated.csv')  # Remplacez par le chemin de votre fichier CSV

# Assurez-vous que votre CSV a deux colonnes 'question' et 'answer'
# Exemple : question, answer
#           سؤال, جواب

# Créer un Dataset HuggingFace à partir du DataFrame
data = {
    'question': df['question_translated'].tolist(),
    'answer': df['answer_translated'].tolist()
}

dataset = Dataset.from_dict(data)

# Diviser le dataset en train et validation (80% train, 20% validation)
train_dataset = dataset.train_test_split(test_size=0.2, seed=42)['train']
val_dataset = dataset.train_test_split(test_size=0.2, seed=42)['test']

# Tokenisation des données
def preprocess_data(examples):
    inputs = [f"سؤال: {q}" for q in examples['question']]
    targets = examples['answer']

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    # Remplacer les labels par un format valide
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in model_inputs["labels"]
    ]
    return model_inputs

# Appliquer la tokenisation sur les datasets d'entraînement et de validation
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/847k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/320M [00:00<?, ?B/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer

# Définir les paramètres d'entraînement
training_args = TrainingArguments(
    output_dir="./results",  # Dossier où sauvegarder le modèle
    evaluation_strategy="epoch",  # Evaluation après chaque époque
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Taille du batch
    num_train_epochs=30,  # Nombre d'époques
    weight_decay=0.01,
    save_strategy="epoch",  # Sauvegarde après chaque époque
    save_steps=None,  # Désactiver la sauvegarde basée sur les étapes
    logging_dir="./logs",  # Dossier de logs
    logging_steps=10,
    load_best_model_at_end=True,  # Charger le meilleur modèle à la fin de l'entraînement
    metric_for_best_model="loss"  # Utiliser la perte pour évaluer le meilleur modèle
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Ajouter le dataset de validation ici
    tokenizer=tokenizer,
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle fine-tuné
model.save_pretrained("./fine_tuned_arabic_t5")
tokenizer.save_pretrained("./fine_tuned_arabic_t5")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-671b8fd86eca>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,23.537600,22.136620
2,22.830200,18.508808
3,19.844500,16.516294
4,18.469200,14.020518
5,18.596800,12.256925
6,16.833500,11.450153
7,15.562000,10.325069
8,15.365600,9.914960
9,14.604300,9.689516
10,13.725200,9.096728


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


('./fine_tuned_arabic_t5/tokenizer_config.json',
 './fine_tuned_arabic_t5/special_tokens_map.json',
 './fine_tuned_arabic_t5/spiece.model',
 './fine_tuned_arabic_t5/added_tokens.json',
 './fine_tuned_arabic_t5/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Charger le modèle et le tokenizer
tokenizer = AutoTokenizer.from_pretrained("bakrianoo/t5-arabic-small")
model = AutoModelForSeq2SeqLM.from_pretrained("bakrianoo/t5-arabic-small")

# Vérifier si CUDA est disponible et utiliser le GPU si possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Déplacer le modèle sur le même dispositif (GPU ou CPU)
model.to(device)

def generate_answer(question):
    # Essayer sans préfixe ou avec un autre format
    input_text = f"{question}"

    # Tokenisation de la question
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Déplacer les input_ids sur le même dispositif que le modèle
    input_ids = input_ids.to(device)

    # Génération de la réponse
    output_ids = model.generate(
    input_ids,
    max_length=128,        # Réduire la longueur maximale
    num_beams=3,           # Réduire le nombre de beams
    early_stopping=True,
    no_repeat_ngram_size=2,
    do_sample=True,        # Essayer l'échantillonnage
    top_p=0.9,             # Ajuster le top_p
    top_k=30               # Ajuster le top_k
      )


    # Décoder la réponse
    decoded_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return decoded_answer


# Tester avec une question
question = "ما هو الذكاء الاصطناعي؟"
answer = generate_answer(question)
print(f"Réponse: {answer}")


# Exemple de question pour tester
questions = [
    "ما هو الفرق الرئيسي الرئيسي بين التعلم تحت الإشراف وغير المنظم للآلات؟	",
    "كيف يعمل التعلم الآلي؟",
    "ما هو الفرق بين الذكاء الاصطناعي والتعلم الآلي؟"
]

# Tester le modèle avec plusieurs questions
for question in questions:
    answer = generate_answer(question)
    print(f"Question: {question}")
    print(f"Réponse: {answer}\n")

question = "ما هو الذكاء الاصطناعي؟"
answer = generate_answer(question)
print(f"Réponse: {answer}")


Réponse: <extra_id_0>
Question: ما هو الفرق الرئيسي الرئيسي بين التعلم تحت الإشراف وغير المنظم للآلات؟	
Réponse: <extra_id_0>.

Question: كيف يعمل التعلم الآلي؟
Réponse: <extra_id_0>

Question: ما هو الفرق بين الذكاء الاصطناعي والتعلم الآلي؟
Réponse: <extra_id_0>

Réponse: <extra_id_0>


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

# Charger les données depuis le fichier CSV
data_path = "questions_translated.csv"  # Remplacez par le chemin de votre fichier CSV
df = pd.read_csv(data_path)

# Préparer les données pour l'entraînement
def create_input_output_pair(row):
    return {
        'input_text': f"السؤال: {row['question_translated']}؟ الجواب: ",
        'target_text': row['answer_translated']
    }

# Appliquer la fonction pour créer le format des paires (input, output)
formatted_data = df.apply(create_input_output_pair, axis=1)

# Convertir en DataFrame avant de le convertir en dataset Hugging Face
formatted_data_df = pd.DataFrame(formatted_data.tolist())

# Créer un Dataset Hugging Face
dataset = Dataset.from_pandas(formatted_data_df)

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-medium")
model = AutoModelForCausalLM.from_pretrained("aubmindlab/aragpt2-medium")

# Ajouter un jeton de padding au tokenizer
tokenizer.pad_token = tokenizer.eos_token  # Vous pouvez définir un autre jeton si vous le souhaitez

# Tokenisation des données avec padding et truncation
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)
    # Les labels sont les mêmes que les inputs (enlever le token [PAD] dans les labels)
    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

# Tokeniser le dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Charger le collateur de données
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Paramètres d'entraînement
training_args = TrainingArguments(
    output_dir='./results',         # Répertoire où les résultats seront sauvegardés
    evaluation_strategy="no",       # Pas d'évaluation pendant l'entraînement
    learning_rate=5e-5,             # Taux d'apprentissage
    per_device_train_batch_size=4,  # Taille du lot
    num_train_epochs=5,             # Nombre d'époques
    weight_decay=0.01,              # Décroissance du poids
)

# Créer l'entraîneur avec le collateur de données
trainer = Trainer(
    model=model,                             # Modèle à entraîner
    args=training_args,                      # Paramètres d'entraînement
    train_dataset=tokenized_datasets,        # Données d'entraînement
    data_collator=data_collator              # Utiliser le collateur de données
)

# Entraîner le modèle
trainer.train()

# Sauvegarder le modèle entraîné
model.save_pretrained('./arabic_question_answer_model')
tokenizer.save_pretrained('./arabic_question_answer_model')


Map:   0%|          | 0/145 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss


('./arabic_question_answer_model/tokenizer_config.json',
 './arabic_question_answer_model/special_tokens_map.json',
 './arabic_question_answer_model/vocab.json',
 './arabic_question_answer_model/merges.txt',
 './arabic_question_answer_model/added_tokens.json',
 './arabic_question_answer_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Charger le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained('./arabic_question_answer_model')
model = AutoModelForCausalLM.from_pretrained('./arabic_question_answer_model')

# Ajouter un pad_token_id si nécessaire
tokenizer.pad_token = tokenizer.eos_token  # Utiliser l'EOS comme token de padding
model.config.pad_token_id = model.config.eos_token_id  # Définir le pad_token_id dans la configuration du modèle

# Tester le modèle avec une question
question = "هل تذكر الاختلافات بين السببية والارتباط؟"
input_text = f"السؤال: {question}؟ الجواب: "

# Encoder la question et ajouter l'attention_mask
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=100)  # Définir max_length

# Générer une réponse
outputs = model.generate(
    inputs["input_ids"],
    max_length=100,  # Limiter la longueur de génération
    attention_mask=inputs["attention_mask"],  # Passer l'attention_mask
)

# Décoder la réponse
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
